### Install necessary packages 
* [ortools](https://developers.google.com/optimization) is an open source optimization tool 

In [ ]:
!python -m pip install --upgrade --user ortools 

### Using 4-vertices clique to demo maxcut algorithm in ortool 

* network visualization 
* graph partition and cut counting   
* objective function 

In [ ]:
import networkx as nx

n_nodes = 4
p = 1  # probability of an edge, exercise, can change it to other probability 
# p = 0.3
seed = 1967

g = nx.erdos_renyi_graph(n_nodes, p=p, seed=seed)
positions = nx.spring_layout(g, seed=seed)

nx.draw(g, with_labels=True, pos=positions, node_size=600)

## Use OR-Tools to solve Max Cut Problem 

* Objective function is quadratic 
* Using CP_Model and intermediate variables to optimize quadratic target function 

In [ ]:
# might needs restart the notebook 
from ortools.sat.python import cp_model

In [ ]:
model = cp_model.CpModel()
x0 = model.NewIntVar(-1, 1, 'x0')
x1 = model.NewIntVar(-1, 1, 'x1')
x2 = model.NewIntVar(-1, 1, 'x2')
x3 = model.NewIntVar(-1, 1, 'x3')


a = model.NewIntVar(-1, 1, 'a')
b = model.NewIntVar(-1, 1, 'b')
c = model.NewIntVar(-1, 1, 'c')
d = model.NewIntVar(-1, 1, 'd')
e = model.NewIntVar(-1, 1, 'e')
f = model.NewIntVar(-1, 1, 'f')


model.AddMultiplicationEquality(a, [x0, x1])
model.AddMultiplicationEquality(b, [x1, x2])
model.AddMultiplicationEquality(c, [x2, x3])
model.AddMultiplicationEquality(d, [x3, x0])
model.AddMultiplicationEquality(e, [x0, x2])
model.AddMultiplicationEquality(f, [x1, x3])




In [ ]:
# model.Maximize( 1-b + 1-c + 1-e ) 
model.Maximize(1-a + 1-b + 1-c + 1-d + 1-e + 1 - f) 
solver = cp_model.CpSolver()
solution_printer = cp_model.VarArrayAndObjectiveSolutionPrinter([x0, x1, x2, x3])
status = solver.Solve(model, solution_printer)



In [ ]:
print (solver.StatusName(status))

### Max cut over n-vertices graph

In [ ]:
import networkx as nx

n_nodes = 25
p = 0.5  # probability of an edge
seed = 1967

g = nx.erdos_renyi_graph(n_nodes, p=p, seed=seed)
positions = nx.spring_layout(g, seed=seed)

nx.draw(g, with_labels=True, pos=positions, node_size=600)

In [ ]:
edges = nx.to_numpy_matrix(g)

In [ ]:
edges = edges.tolist()
edges

In [ ]:
model = cp_model.CpModel()

nodes = [None for i in range(0, n_nodes)] 
for i in range(0, n_nodes): 
    name = "x"+str(i)
    nodes[i] = model.NewIntVar(-1, 1, name)

optvar = [[None for i in range(0, n_nodes)] for j in range(0, n_nodes)]     
for j in range(0, n_nodes):
    for i in range(0, n_nodes):
        if j > i:
            name = "x{}x{}".format(str(i), str(j))
            optvar[i][j] = model.NewIntVar(-1, 1, name)
            model.AddMultiplicationEquality( optvar[i][j], [nodes[i], nodes[j]])
    


In [ ]:
def objective(optvar, edges, n_nodes):
    exp = None 
    for j in range(0, n_nodes): 
        for i in range(0, n_nodes): 
            if j > i and edges[i][j] > 0: 
                if exp == None: 
                    exp = (1 - optvar[i][j]*int(edges[i][j]))
                else:
                    exp += (1 - optvar[i][j]*int(edges[i][j])) 
    return exp         

In [ ]:
model.Maximize(objective(optvar, edges, n_nodes))

In [ ]:
%%time
solver = cp_model.CpSolver()
solution_printer = cp_model.VarArrayAndObjectiveSolutionPrinter(nodes)
solver.parameters.num_search_workers = 12
status = solver.Solve(model, solution_printer)